In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os
import json
import logging
warnings.filterwarnings('ignore')

# Setup logging for GCP
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class StockDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = torch.FloatTensor(sequences)
        self.targets = torch.FloatTensor(targets)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

class LSTMPredictor(nn.Module):
    def __init__(self, input_size=10, hidden_size=32, num_layers=2, output_size=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 16),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(16, output_size),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])

class MultiStockPredictor:
    def __init__(self, symbol='AAPL', data_period='2y', lookback_days=15):
        self.symbol = symbol.upper()
        self.data_period = data_period
        self.lookback_days = lookback_days
        self.scaler = StandardScaler()
        self.price_scaler = MinMaxScaler()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Initialize models
        self.models = {}
        self.results = {}
        
        logger.info(f"Initialized predictor for {self.symbol}")
    
    def fetch_and_prepare_data(self):
        """Fetch data using yfinance and prepare features"""
        logger.info(f"Fetching data for {self.symbol}")
        
        ticker = yf.Ticker(self.symbol)
        hist = ticker.history(period=self.data_period)
        
        if hist.empty:
            raise ValueError(f"No data available for {self.symbol}")
        
        # Calculate technical indicators
        hist = self._calculate_indicators(hist)
        
        # Save to CSV for later use
        hist.to_csv(f"{self.symbol}_technical_analysis.csv")
        
        self.current_price = hist['Close'].iloc[-1]
        logger.info(f"Current {self.symbol} price: ${self.current_price:.2f}")
        
        return hist
    
    def _calculate_indicators(self, df):
        """Calculate comprehensive technical indicators"""
        # Price-based indicators
        df['SMA_5'] = df['Close'].rolling(5).mean()
        df['SMA_10'] = df['Close'].rolling(10).mean()
        df['SMA_20'] = df['Close'].rolling(20).mean()
        df['EMA_12'] = df['Close'].ewm(span=12).mean()
        df['EMA_26'] = df['Close'].ewm(span=26).mean()
        
        # MACD
        df['MACD'] = df['EMA_12'] - df['EMA_26']
        df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
        df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
        
        # RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        # Bollinger Bands
        bb_period = 20
        df['BB_Mid'] = df['Close'].rolling(bb_period).mean()
        bb_std = df['Close'].rolling(bb_period).std()
        df['BB_Upper'] = df['BB_Mid'] + (bb_std * 2)
        df['BB_Lower'] = df['BB_Mid'] - (bb_std * 2)
        df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Mid']
        df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
        
        # Volume indicators
        df['Volume_SMA'] = df['Volume'].rolling(20).mean()
        df['Volume_Ratio'] = df['Volume'] / df['Volume_SMA']
        
        # Volatility
        df['Returns'] = df['Close'].pct_change()
        df['Volatility'] = df['Returns'].rolling(10).std()
        
        # Price momentum
        df['Price_Change_5d'] = df['Close'].pct_change(5)
        df['Price_Change_10d'] = df['Close'].pct_change(10)
        
        # Support/Resistance
        df['High_20d'] = df['High'].rolling(20).max()
        df['Low_20d'] = df['Low'].rolling(20).min()
        df['Price_Position'] = (df['Close'] - df['Low_20d']) / (df['High_20d'] - df['Low_20d'])
        
        return df.fillna(method='ffill').fillna(0)
    
    def create_sequences(self, df, target_col, prediction_days=5):
        """Create sequences for time series prediction"""
        feature_cols = ['Returns', 'RSI', 'MACD', 'BB_Position', 'Volume_Ratio', 
                       'Volatility', 'Price_Change_5d', 'Price_Position', 'MACD_Hist', 'BB_Width']
        
        features = df[feature_cols].values
        targets = df[target_col].values
        
        sequences, target_sequences = [], []
        
        for i in range(self.lookback_days, len(df) - prediction_days):
            sequences.append(features[i-self.lookback_days:i])
            target_sequences.append(targets[i+prediction_days])
        
        return np.array(sequences), np.array(target_sequences)
    
    def predict_price_direction(self, df):
        """Predict if price will go up or down in next 5 days"""
        logger.info("Training price direction prediction model...")
        
        # Create target: 1 if price goes up in 5 days, 0 if down
        df['Future_Price'] = df['Close'].shift(-5)
        df['Direction'] = (df['Future_Price'] > df['Close']).astype(int)
        
        sequences, targets = self.create_sequences(df, 'Direction')
        
        if len(sequences) < 50:
            return {'accuracy': 0.5, 'prediction': 0.5}
        
        # Split data
        split_idx = int(len(sequences) * 0.8)
        X_train, X_test = sequences[:split_idx], sequences[split_idx:]
        y_train, y_test = targets[:split_idx], targets[split_idx:]
        
        # Reshape for sklearn models
        X_train_flat = X_train.reshape(X_train.shape[0], -1)
        X_test_flat = X_test.reshape(X_test.shape[0], -1)
        
        # Train multiple models
        models = {
            'random_forest': RandomForestClassifier(n_estimators=50, random_state=42),
            'gradient_boost': GradientBoostingClassifier(n_estimators=50, random_state=42),
            'logistic': LogisticRegression(random_state=42)
        }
        
        best_model, best_acc = None, 0
        
        for name, model in models.items():
            model.fit(X_train_flat, y_train)
            y_pred = model.predict(X_test_flat)
            acc = accuracy_score(y_test, y_pred)
            
            if acc > best_acc:
                best_acc = acc
                best_model = model
        
        # Make prediction on latest data
        latest_seq = sequences[-1:].reshape(1, -1)
        prediction = best_model.predict_proba(latest_seq)[0][1]
        
        return {
            'model_type': 'direction_classifier',
            'accuracy': best_acc,
            'prediction': prediction,
            'signal': 'BUY' if prediction > 0.6 else 'SELL' if prediction < 0.4 else 'HOLD'
        }
    
    def predict_volatility_breakout(self, df):
        """Predict if volatility will increase significantly"""
        logger.info("Training volatility breakout prediction...")
        
        # Target: 1 if volatility increases by 50% in next 3 days
        df['Future_Vol'] = df['Volatility'].shift(-3)
        df['Vol_Breakout'] = (df['Future_Vol'] > df['Volatility'] * 1.5).astype(int)
        
        sequences, targets = self.create_sequences(df, 'Vol_Breakout', 3)
        
        if len(sequences) < 30:
            return {'accuracy': 0.5, 'prediction': 0.5}
        
        # Train LSTM for this task
        model = LSTMPredictor(input_size=sequences.shape[-1], hidden_size=16)
        
        # Simple training loop
        sequences_scaled = self.scaler.fit_transform(sequences.reshape(-1, sequences.shape[-1])).reshape(sequences.shape)
        dataset = StockDataset(sequences_scaled, targets)
        loader = DataLoader(dataset, batch_size=8, shuffle=True)
        
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.BCELoss()
        
        for epoch in range(20):
            for batch_seq, batch_targets in loader:
                optimizer.zero_grad()
                outputs = model(batch_seq).squeeze()
                loss = criterion(outputs, batch_targets)
                loss.backward()
                optimizer.step()
        
        # Predict latest
        latest_scaled = self.scaler.transform(sequences[-1:].reshape(-1, sequences.shape[-1])).reshape(1, self.lookback_days, -1)
        model.eval()
        with torch.no_grad():
            prediction = model(torch.FloatTensor(latest_scaled)).item()
        
        return {
            'model_type': 'volatility_lstm',
            'prediction': prediction,
            'signal': 'HIGH_VOL_EXPECTED' if prediction > 0.7 else 'NORMAL_VOL'
        }
    
    def predict_mean_reversion(self, df):
        """Predict mean reversion opportunities"""
        logger.info("Training mean reversion prediction...")
        
        # Target: price returns to 20-day SMA within 7 days
        df['Distance_SMA'] = (df['Close'] - df['SMA_20']) / df['SMA_20']
        df['Future_Distance'] = df['Distance_SMA'].shift(-7)
        
        # Mean reversion: if currently far from SMA, will it return?
        df['Will_Revert'] = ((abs(df['Distance_SMA']) > 0.05) & 
                            (abs(df['Future_Distance']) < abs(df['Distance_SMA']))).astype(int)
        
        sequences, targets = self.create_sequences(df, 'Will_Revert', 7)
        
        if len(sequences) < 30:
            return {'prediction': 0.5}
        
        # Use Gradient Boosting for mean reversion
        X = sequences.reshape(sequences.shape[0], -1)
        model = GradientBoostingClassifier(n_estimators=30, random_state=42)
        model.fit(X, targets)
        
        latest_seq = sequences[-1:].reshape(1, -1)
        prediction = model.predict_proba(latest_seq)[0][1]
        current_distance = df['Distance_SMA'].iloc[-1]
        
        return {
            'model_type': 'mean_reversion_gb',
            'prediction': prediction,
            'current_distance_from_sma': current_distance,
            'signal': 'MEAN_REVERSION' if abs(current_distance) > 0.05 and prediction > 0.6 else 'NO_REVERSION'
        }
    
    def predict_momentum_continuation(self, df):
        """Predict if current momentum will continue"""
        logger.info("Training momentum continuation prediction...")
        
        # Target: if current 5-day trend continues for another 5 days
        df['Current_Trend'] = np.sign(df['Price_Change_5d'])
        df['Future_Trend'] = np.sign(df['Price_Change_5d'].shift(-5))
        df['Momentum_Continues'] = (df['Current_Trend'] == df['Future_Trend']).astype(int)
        
        sequences, targets = self.create_sequences(df, 'Momentum_Continues')
        
        if len(sequences) < 30:
            return {'prediction': 0.5}
        
        # Random Forest for momentum
        X = sequences.reshape(sequences.shape[0], -1)
        model = RandomForestClassifier(n_estimators=40, random_state=42)
        model.fit(X, targets)
        
        prediction = model.predict_proba(sequences[-1:].reshape(1, -1))[0][1]
        current_trend = df['Current_Trend'].iloc[-1]
        
        return {
            'model_type': 'momentum_rf',
            'prediction': prediction,
            'current_trend': 'UP' if current_trend > 0 else 'DOWN' if current_trend < 0 else 'NEUTRAL',
            'signal': 'MOMENTUM_CONTINUE' if prediction > 0.65 else 'MOMENTUM_REVERSAL'
        }
    
    def predict_options_implied_move(self, df):
        """Predict if stock will move more than historical volatility suggests"""
        logger.info("Training implied move prediction...")
        
        # Simplified options prediction based on volatility patterns
        df['Vol_Percentile'] = df['Volatility'].rolling(252).rank(pct=True)
        df['Future_Large_Move'] = (abs(df['Returns'].shift(-3)) > df['Volatility'] * 1.5).astype(int)
        
        sequences, targets = self.create_sequences(df, 'Future_Large_Move', 3)
        
        if len(sequences) < 30:
            return {'prediction': 0.3}
        
        model = LogisticRegression()
        X = sequences.reshape(sequences.shape[0], -1)
        model.fit(X, targets)
        
        prediction = model.predict_proba(sequences[-1:].reshape(1, -1))[0][1]
        current_vol_percentile = df['Vol_Percentile'].iloc[-1]
        
        return {
            'model_type': 'implied_move_logistic',
            'prediction': prediction,
            'vol_percentile': current_vol_percentile,
            'signal': 'LARGE_MOVE_EXPECTED' if prediction > 0.6 else 'NORMAL_MOVE'
        }
    
    def create_ensemble_prediction(self):
        """Combine all predictions into ensemble forecast"""
        predictions = []
        weights = []
        
        for name, result in self.results.items():
            if 'prediction' in result:
                predictions.append(result['prediction'])
                # Weight by accuracy if available
                weight = result.get('accuracy', 0.5)
                weights.append(weight)
        
        if not predictions:
            return 0.5
        
        # Weighted average
        weights = np.array(weights)
        weights = weights / weights.sum()
        ensemble = np.average(predictions, weights=weights)
        
        return {
            'ensemble_prediction': ensemble,
            'confidence': np.std(predictions),  # Lower std = higher confidence
            'signal': 'STRONG_BUY' if ensemble > 0.7 else 'BUY' if ensemble > 0.6 else 
                     'STRONG_SELL' if ensemble < 0.3 else 'SELL' if ensemble < 0.4 else 'HOLD'
        }
    
    def run_all_predictions(self):
        """Execute all prediction models"""
        try:
            # Fetch and prepare data
            df = self.fetch_and_prepare_data()
            
            # Run all prediction models
            logger.info("Running all prediction models...")
            
            self.results['direction'] = self.predict_price_direction(df)
            self.results['volatility'] = self.predict_volatility_breakout(df)
            self.results['mean_reversion'] = self.predict_mean_reversion(df)
            self.results['momentum'] = self.predict_momentum_continuation(df)
            self.results['implied_move'] = self.predict_options_implied_move(df)
            
            # Create ensemble
            self.results['ensemble'] = self.create_ensemble_prediction()
            
            # Add summary info
            self.results['summary'] = {
                'symbol': self.symbol,
                'current_price': self.current_price,
                'analysis_date': datetime.now().isoformat(),
                'total_predictions': len([r for r in self.results.values() if 'prediction' in r])
            }
            
            return self.results
            
        except Exception as e:
            logger.error(f"Error in predictions: {e}")
            return {'error': str(e)}
    
    def save_results(self, filename=None):
        """Save results to JSON file"""
        if filename is None:
            filename = f"{self.symbol}_predictions_{datetime.now().strftime('%Y%m%d_%H%M')}.json"
        
        with open(filename, 'w') as f:
            json.dump(self.results, f, indent=2, default=str)
        
        logger.info(f"Results saved to {filename}")
    
    def display_results(self):
        """Display formatted results"""
        print(f"\n{'='*60}")
        print(f"MULTI-MODEL STOCK PREDICTIONS - {self.symbol}")
        print(f"Current Price: ${self.current_price:.2f}")
        print(f"{'='*60}")
        
        for name, result in self.results.items():
            if name == 'summary':
                continue
                
            print(f"\n📊 {name.upper()} PREDICTION:")
            if 'prediction' in result:
                print(f"  Probability: {result['prediction']:.3f}")
            if 'signal' in result:
                print(f"  Signal: {result['signal']}")
            if 'accuracy' in result:
                print(f"  Model Accuracy: {result['accuracy']:.3f}")
        
        if 'ensemble' in self.results:
            ens = self.results['ensemble']
            print(f"\n🎯 ENSEMBLE FORECAST:")
            print(f"  Combined Prediction: {ens['ensemble_prediction']:.3f}")
            print(f"  Confidence: {1-ens['confidence']:.3f}")
            print(f"  Final Signal: {ens['signal']}")

# GCP Optimization Functions (Additional 50 lines)
def setup_gcp_logging():
    """Setup Google Cloud Logging"""
    try:
        from google.cloud import logging as cloud_logging
        client = cloud_logging.Client()
        client.setup_logging()
        return client
    except ImportError:
        logger.info("Google Cloud Logging not available, using standard logging")
        return None

def save_to_gcs(data, bucket_name, blob_name):
    """Save data to Google Cloud Storage"""
    try:
        from google.cloud import storage
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        blob.upload_from_string(json.dumps(data, default=str))
        logger.info(f"Data saved to gs://{bucket_name}/{blob_name}")
    except ImportError:
        logger.info("Google Cloud Storage not available")

def cloud_function_handler(request):
    """Google Cloud Function entry point"""
    try:
        request_json = request.get_json(silent=True)
        symbol = request_json.get('symbol', 'AAPL') if request_json else 'AAPL'
        
        predictor = MultiStockPredictor(symbol=symbol)
        results = predictor.run_all_predictions()
        
        # Save to GCS if configured
        bucket_name = os.environ.get('GCS_BUCKET')
        if bucket_name:
            blob_name = f"predictions/{symbol}_{datetime.now().strftime('%Y%m%d_%H%M')}.json"
            save_to_gcs(results, bucket_name, blob_name)
        
        return results
    except Exception as e:
        logger.error(f"Cloud function error: {e}")
        return {'error': str(e)}

def batch_predict_multiple_stocks(symbols, max_workers=3):
    """Batch prediction for multiple stocks (GCP optimized)"""
    import concurrent.futures
    
    def predict_single(symbol):
        predictor = MultiStockPredictor(symbol=symbol)
        return symbol, predictor.run_all_predictions()
    
    results = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(predict_single, symbol): symbol for symbol in symbols}
        
        for future in concurrent.futures.as_completed(futures):
            symbol = futures[future]
            try:
                symbol, result = future.result()
                results[symbol] = result
            except Exception as e:
                logger.error(f"Error predicting {symbol}: {e}")
                results[symbol] = {'error': str(e)}
    
    return results

# Main execution
if __name__ == "__main__":
    # Setup GCP logging if available
    gcp_logger = setup_gcp_logging()
    
    # Single stock prediction
    SYMBOL = "AAPL"
    
    predictor = MultiStockPredictor(symbol=SYMBOL)
    results = predictor.run_all_predictions()
    
    predictor.display_results()
    predictor.save_results()
    
    # Multi-stock example
    # symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA']
    # batch_results = batch_predict_multiple_stocks(symbols)
    # print(f"Batch prediction completed for {len(batch_results)} stocks")

INFO:__main__:Google Cloud Logging not available, using standard logging
INFO:__main__:Initialized predictor for AAPL
INFO:__main__:Fetching data for AAPL
INFO:__main__:Current AAPL price: $238.15
INFO:__main__:Running all prediction models...
INFO:__main__:Training price direction prediction model...
INFO:__main__:Training volatility breakout prediction...
INFO:__main__:Training mean reversion prediction...
INFO:__main__:Training momentum continuation prediction...
INFO:__main__:Training implied move prediction...
INFO:__main__:Results saved to AAPL_predictions_20250916_1800.json



MULTI-MODEL STOCK PREDICTIONS - AAPL
Current Price: $238.15

📊 DIRECTION PREDICTION:
  Probability: 0.280
  Signal: SELL
  Model Accuracy: 0.536

📊 VOLATILITY PREDICTION:
  Probability: 0.043
  Signal: NORMAL_VOL

📊 MEAN_REVERSION PREDICTION:
  Probability: 0.160
  Signal: NO_REVERSION

📊 MOMENTUM PREDICTION:
  Probability: 0.175
  Signal: MOMENTUM_REVERSAL

📊 IMPLIED_MOVE PREDICTION:
  Probability: 0.069
  Signal: NORMAL_MOVE

📊 ENSEMBLE PREDICTION:
  Signal: STRONG_SELL

🎯 ENSEMBLE FORECAST:
  Combined Prediction: 0.147
  Confidence: 0.916
  Final Signal: STRONG_SELL


In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os
import json
import logging
warnings.filterwarnings('ignore')

# Setup logging for GCP
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class StockDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = torch.FloatTensor(sequences)
        self.targets = torch.FloatTensor(targets)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

class LSTMPredictor(nn.Module):
    def __init__(self, input_size=10, hidden_size=32, num_layers=2, output_size=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 16),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(16, output_size),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])

class MultiStockPredictor:
    def __init__(self, symbol='AAPL', data_period='2y', lookback_days=15):
        self.symbol = symbol.upper()
        self.data_period = data_period
        self.lookback_days = lookback_days
        self.scaler = StandardScaler()
        self.price_scaler = MinMaxScaler()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Initialize models
        self.models = {}
        self.results = {}
        
        logger.info(f"Initialized predictor for {self.symbol}")
    
    def fetch_and_prepare_data(self):
        """Fetch data using yfinance and prepare features"""
        logger.info(f"Fetching data for {self.symbol}")
        
        ticker = yf.Ticker(self.symbol)
        hist = ticker.history(period=self.data_period)
        
        if hist.empty:
            raise ValueError(f"No data available for {self.symbol}")
        
        # Calculate technical indicators
        hist = self._calculate_indicators(hist)
        
        # Save to CSV for later use
        hist.to_csv(f"{self.symbol}_technical_analysis.csv")
        
        self.current_price = hist['Close'].iloc[-1]
        logger.info(f"Current {self.symbol} price: ${self.current_price:.2f}")
        
        return hist
    
    def _calculate_indicators(self, df):
        """Calculate comprehensive technical indicators"""
        # Price-based indicators
        df['SMA_5'] = df['Close'].rolling(5).mean()
        df['SMA_10'] = df['Close'].rolling(10).mean()
        df['SMA_20'] = df['Close'].rolling(20).mean()
        df['EMA_12'] = df['Close'].ewm(span=12).mean()
        df['EMA_26'] = df['Close'].ewm(span=26).mean()
        
        # MACD
        df['MACD'] = df['EMA_12'] - df['EMA_26']
        df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
        df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
        
        # RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        # Bollinger Bands
        bb_period = 20
        df['BB_Mid'] = df['Close'].rolling(bb_period).mean()
        bb_std = df['Close'].rolling(bb_period).std()
        df['BB_Upper'] = df['BB_Mid'] + (bb_std * 2)
        df['BB_Lower'] = df['BB_Mid'] - (bb_std * 2)
        df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Mid']
        df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
        
        # Volume indicators
        df['Volume_SMA'] = df['Volume'].rolling(20).mean()
        df['Volume_Ratio'] = df['Volume'] / df['Volume_SMA']
        
        # Volatility
        df['Returns'] = df['Close'].pct_change()
        df['Volatility'] = df['Returns'].rolling(10).std()
        
        # Price momentum
        df['Price_Change_5d'] = df['Close'].pct_change(5)
        df['Price_Change_10d'] = df['Close'].pct_change(10)
        
        # Support/Resistance
        df['High_20d'] = df['High'].rolling(20).max()
        df['Low_20d'] = df['Low'].rolling(20).min()
        df['Price_Position'] = (df['Close'] - df['Low_20d']) / (df['High_20d'] - df['Low_20d'])
        
        return df.fillna(method='ffill').fillna(0)
    
    def create_sequences(self, df, target_col, prediction_days=5):
        """Create sequences for time series prediction"""
        feature_cols = ['Returns', 'RSI', 'MACD', 'BB_Position', 'Volume_Ratio', 
                       'Volatility', 'Price_Change_5d', 'Price_Position', 'MACD_Hist', 'BB_Width']
        
        features = df[feature_cols].values
        targets = df[target_col].values
        
        sequences, target_sequences = [], []
        
        for i in range(self.lookback_days, len(df) - prediction_days):
            sequences.append(features[i-self.lookback_days:i])
            target_sequences.append(targets[i+prediction_days])
        
        return np.array(sequences), np.array(target_sequences)
    
    def predict_price_direction(self, df):
        """Predict if price will go up or down in next 5 days"""
        logger.info("Training price direction prediction model...")
        
        # Create target: 1 if price goes up in 5 days, 0 if down
        df['Future_Price'] = df['Close'].shift(-5)
        df['Direction'] = (df['Future_Price'] > df['Close']).astype(int)
        
        sequences, targets = self.create_sequences(df, 'Direction')
        
        if len(sequences) < 50:
            return {'accuracy': 0.5, 'prediction': 0.5}
        
        # Split data
        split_idx = int(len(sequences) * 0.8)
        X_train, X_test = sequences[:split_idx], sequences[split_idx:]
        y_train, y_test = targets[:split_idx], targets[split_idx:]
        
        # Reshape for sklearn models
        X_train_flat = X_train.reshape(X_train.shape[0], -1)
        X_test_flat = X_test.reshape(X_test.shape[0], -1)
        
        # Train multiple models
        models = {
            'random_forest': RandomForestClassifier(n_estimators=50, random_state=42),
            'gradient_boost': GradientBoostingClassifier(n_estimators=50, random_state=42),
            'logistic': LogisticRegression(random_state=42)
        }
        
        best_model, best_acc = None, 0
        
        for name, model in models.items():
            model.fit(X_train_flat, y_train)
            y_pred = model.predict(X_test_flat)
            acc = accuracy_score(y_test, y_pred)
            
            if acc > best_acc:
                best_acc = acc
                best_model = model
        
        # Make prediction on latest data
        latest_seq = sequences[-1:].reshape(1, -1)
        prediction = best_model.predict_proba(latest_seq)[0][1]
        
        return {
            'model_type': 'direction_classifier',
            'accuracy': best_acc,
            'prediction': prediction,
            'signal': 'BUY' if prediction > 0.6 else 'SELL' if prediction < 0.4 else 'HOLD'
        }
    
    def predict_volatility_breakout(self, df):
        """Predict if volatility will increase significantly"""
        logger.info("Training volatility breakout prediction...")
        
        # Target: 1 if volatility increases by 50% in next 3 days
        df['Future_Vol'] = df['Volatility'].shift(-3)
        df['Vol_Breakout'] = (df['Future_Vol'] > df['Volatility'] * 1.5).astype(int)
        
        sequences, targets = self.create_sequences(df, 'Vol_Breakout', 3)
        
        if len(sequences) < 30:
            return {'accuracy': 0.5, 'prediction': 0.5}
        
        # Train LSTM for this task
        model = LSTMPredictor(input_size=sequences.shape[-1], hidden_size=16)
        
        # Simple training loop
        sequences_scaled = self.scaler.fit_transform(sequences.reshape(-1, sequences.shape[-1])).reshape(sequences.shape)
        dataset = StockDataset(sequences_scaled, targets)
        loader = DataLoader(dataset, batch_size=8, shuffle=True)
        
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.BCELoss()
        
        for epoch in range(20):
            for batch_seq, batch_targets in loader:
                optimizer.zero_grad()
                outputs = model(batch_seq).squeeze()
                loss = criterion(outputs, batch_targets)
                loss.backward()
                optimizer.step()
        
        # Predict latest
        latest_scaled = self.scaler.transform(sequences[-1:].reshape(-1, sequences.shape[-1])).reshape(1, self.lookback_days, -1)
        model.eval()
        with torch.no_grad():
            prediction = model(torch.FloatTensor(latest_scaled)).item()
        
        return {
            'model_type': 'volatility_lstm',
            'prediction': prediction,
            'signal': 'HIGH_VOL_EXPECTED' if prediction > 0.7 else 'NORMAL_VOL'
        }
    
    def predict_mean_reversion(self, df):
        """Predict mean reversion opportunities"""
        logger.info("Training mean reversion prediction...")
        
        # Target: price returns to 20-day SMA within 7 days
        df['Distance_SMA'] = (df['Close'] - df['SMA_20']) / df['SMA_20']
        df['Future_Distance'] = df['Distance_SMA'].shift(-7)
        
        # Mean reversion: if currently far from SMA, will it return?
        df['Will_Revert'] = ((abs(df['Distance_SMA']) > 0.05) & 
                            (abs(df['Future_Distance']) < abs(df['Distance_SMA']))).astype(int)
        
        sequences, targets = self.create_sequences(df, 'Will_Revert', 7)
        
        if len(sequences) < 30:
            return {'prediction': 0.5}
        
        # Use Gradient Boosting for mean reversion
        X = sequences.reshape(sequences.shape[0], -1)
        model = GradientBoostingClassifier(n_estimators=30, random_state=42)
        model.fit(X, targets)
        
        latest_seq = sequences[-1:].reshape(1, -1)
        prediction = model.predict_proba(latest_seq)[0][1]
        current_distance = df['Distance_SMA'].iloc[-1]
        
        return {
            'model_type': 'mean_reversion_gb',
            'prediction': prediction,
            'current_distance_from_sma': current_distance,
            'signal': 'MEAN_REVERSION' if abs(current_distance) > 0.05 and prediction > 0.6 else 'NO_REVERSION'
        }
    
    def predict_momentum_continuation(self, df):
        """Predict if current momentum will continue"""
        logger.info("Training momentum continuation prediction...")
        
        # Target: if current 5-day trend continues for another 5 days
        df['Current_Trend'] = np.sign(df['Price_Change_5d'])
        df['Future_Trend'] = np.sign(df['Price_Change_5d'].shift(-5))
        df['Momentum_Continues'] = (df['Current_Trend'] == df['Future_Trend']).astype(int)
        
        sequences, targets = self.create_sequences(df, 'Momentum_Continues')
        
        if len(sequences) < 30:
            return {'prediction': 0.5}
        
        # Random Forest for momentum
        X = sequences.reshape(sequences.shape[0], -1)
        model = RandomForestClassifier(n_estimators=40, random_state=42)
        model.fit(X, targets)
        
        prediction = model.predict_proba(sequences[-1:].reshape(1, -1))[0][1]
        current_trend = df['Current_Trend'].iloc[-1]
        
        return {
            'model_type': 'momentum_rf',
            'prediction': prediction,
            'current_trend': 'UP' if current_trend > 0 else 'DOWN' if current_trend < 0 else 'NEUTRAL',
            'signal': 'MOMENTUM_CONTINUE' if prediction > 0.65 else 'MOMENTUM_REVERSAL'
        }
    
    def predict_options_implied_move(self, df):
        """Predict if stock will move more than historical volatility suggests"""
        logger.info("Training implied move prediction...")
        
        # Simplified options prediction based on volatility patterns
        df['Vol_Percentile'] = df['Volatility'].rolling(252).rank(pct=True)
        df['Future_Large_Move'] = (abs(df['Returns'].shift(-3)) > df['Volatility'] * 1.5).astype(int)
        
        sequences, targets = self.create_sequences(df, 'Future_Large_Move', 3)
        
        if len(sequences) < 30:
            return {'prediction': 0.3}
        
        model = LogisticRegression()
        X = sequences.reshape(sequences.shape[0], -1)
        model.fit(X, targets)
        
        prediction = model.predict_proba(sequences[-1:].reshape(1, -1))[0][1]
        current_vol_percentile = df['Vol_Percentile'].iloc[-1]
        
        return {
            'model_type': 'implied_move_logistic',
            'prediction': prediction,
            'vol_percentile': current_vol_percentile,
            'signal': 'LARGE_MOVE_EXPECTED' if prediction > 0.6 else 'NORMAL_MOVE'
        }
    
    def predict_specific_price_targets(self, df):
        """Predict probability of reaching specific price targets in 2 and 4 Fridays"""
        logger.info("Training specific price target predictions...")
        
        current_price = df['Close'].iloc[-1]
        
        # Calculate next 2 Fridays and 4 Fridays from current date
        today = datetime.now()
        days_until_friday = (4 - today.weekday()) % 7  # Friday is weekday 4
        if days_until_friday == 0 and today.weekday() == 4:  # If today is Friday
            days_until_friday = 7
        
        friday_2_weeks = today + timedelta(days=days_until_friday + 7)
        friday_4_weeks = today + timedelta(days=days_until_friday + 21)
        
        # Create training data for different time horizons
        target_predictions = {}
        
        # Price ranges: -5% to +5% in 1% increments
        price_targets = []
        for pct_change in range(-5, 6):  # -5% to +5%
            target_price = current_price * (1 + pct_change/100)
            price_targets.append({
                'pct_change': pct_change,
                'target_price': target_price,
                'price_range': f"{pct_change:+d}%"
            })
        
        # Train models for 2-week and 4-week predictions
        for weeks, label in [(2, '2_weeks'), (4, '4_weeks')]:
            trading_days = weeks * 5  # Approximate trading days
            
            # Create targets for each price level
            week_predictions = {}
            
            for target in price_targets:
                pct_change = target['pct_change']
                target_price = target['target_price']
                
                # Create binary target: will stock reach this level?
                if pct_change > 0:
                    # For upward targets: will max price in period exceed target?
                    df[f'Target_{pct_change}'] = (df['High'].shift(-trading_days).rolling(trading_days).max() >= target_price).astype(int)
                elif pct_change < 0:
                    # For downward targets: will min price in period go below target?
                    df[f'Target_{pct_change}'] = (df['Low'].shift(-trading_days).rolling(trading_days).min() <= target_price).astype(int)
                else:
                    # 0% change - will price stay within ±0.5%?
                    upper_bound = current_price * 1.005
                    lower_bound = current_price * 0.995
                    df['Target_0'] = ((df['Close'].shift(-trading_days) >= lower_bound) & 
                                    (df['Close'].shift(-trading_days) <= upper_bound)).astype(int)
                
                # Train model for this target
                try:
                    sequences, targets = self.create_sequences(df, f'Target_{pct_change}', trading_days)
                    
                    if len(sequences) >= 20:
                        # Use ensemble of models
                        X = sequences.reshape(sequences.shape[0], -1)
                        
                        # Train multiple models and average
                        models = [
                            RandomForestClassifier(n_estimators=30, random_state=42),
                            GradientBoostingClassifier(n_estimators=30, random_state=42),
                            LogisticRegression(random_state=42, max_iter=200)
                        ]
                        
                        predictions = []
                        for model in models:
                            model.fit(X, targets)
                            pred = model.predict_proba(X[-1:].reshape(1, -1))[0][1]
                            predictions.append(pred)
                        
                        # Average the predictions
                        avg_prediction = np.mean(predictions)
                        historical_success = np.mean(targets) if len(targets) > 0 else 0.5
                        
                        week_predictions[pct_change] = {
                            'probability': avg_prediction * 100,
                            'target_price': target_price,
                            'historical_success_rate': historical_success * 100,
                            'confidence': 1 - np.std(predictions)  # Lower std = higher confidence
                        }
                    else:
                        # Insufficient data, use historical average
                        week_predictions[pct_change] = {
                            'probability': 50.0,  # Default 50%
                            'target_price': target_price,
                            'historical_success_rate': 50.0,
                            'confidence': 0.3
                        }
                        
                except Exception as e:
                    # Fallback for any errors
                    week_predictions[pct_change] = {
                        'probability': 50.0,
                        'target_price': target_price,
                        'historical_success_rate': 50.0,
                        'confidence': 0.3
                    }
            
            target_predictions[label] = {
                'target_date': friday_2_weeks.strftime('%Y-%m-%d') if weeks == 2 else friday_4_weeks.strftime('%Y-%m-%d'),
                'trading_days': trading_days,
                'predictions': week_predictions
            }
        
        return target_predictions
    
    def create_ensemble_prediction(self):
        """Combine all predictions into ensemble forecast"""
        predictions = []
        weights = []
        
        for name, result in self.results.items():
            if 'prediction' in result:
                predictions.append(result['prediction'])
                # Weight by accuracy if available
                weight = result.get('accuracy', 0.5)
                weights.append(weight)
        
        if not predictions:
            return 0.5
        
        # Weighted average
        weights = np.array(weights)
        weights = weights / weights.sum()
        ensemble = np.average(predictions, weights=weights)
        
        return {
            'ensemble_prediction': ensemble,
            'confidence': np.std(predictions),  # Lower std = higher confidence
            'signal': 'STRONG_BUY' if ensemble > 0.7 else 'BUY' if ensemble > 0.6 else 
                     'STRONG_SELL' if ensemble < 0.3 else 'SELL' if ensemble < 0.4 else 'HOLD'
        }
    
    def run_all_predictions(self):
        """Execute all prediction models"""
        try:
            # Fetch and prepare data
            df = self.fetch_and_prepare_data()
            
            # Run all prediction models
            logger.info("Running all prediction models...")
            
            self.results['direction'] = self.predict_price_direction(df)
            self.results['volatility'] = self.predict_volatility_breakout(df)
            self.results['mean_reversion'] = self.predict_mean_reversion(df)
            self.results['momentum'] = self.predict_momentum_continuation(df)
            self.results['implied_move'] = self.predict_options_implied_move(df)
            self.results['price_targets'] = self.predict_specific_price_targets(df)
            
            # Create ensemble
            self.results['ensemble'] = self.create_ensemble_prediction()
            
            # Add summary info
            self.results['summary'] = {
                'symbol': self.symbol,
                'current_price': self.current_price,
                'analysis_date': datetime.now().isoformat(),
                'total_predictions': len([r for r in self.results.values() if 'prediction' in r])
            }
            
            return self.results
            
        except Exception as e:
            logger.error(f"Error in predictions: {e}")
            return {'error': str(e)}
    
    def save_results(self, filename=None):
        """Save results to JSON and CSV files"""
        if filename is None:
            filename = f"{self.symbol}_predictions_{datetime.now().strftime('%Y%m%d_%H%M')}"
        
        # Save JSON
        with open(f"{filename}.json", 'w') as f:
            json.dump(self.results, f, indent=2, default=str)
        
        # Save detailed CSV if not using GCP
        self.save_detailed_csv(f"{filename}_detailed.csv")
        
        logger.info(f"Results saved to {filename}.json and {filename}_detailed.csv")
    
    def save_detailed_csv(self, filename):
        """Save detailed results to CSV format"""
        rows = []
        
        # Add main predictions
        for name, result in self.results.items():
            if name in ['summary', 'price_targets']:
                continue
            
            row = {
                'prediction_type': name,
                'probability': result.get('prediction', 0),
                'signal': result.get('signal', 'N/A'),
                'accuracy': result.get('accuracy', 'N/A'),
                'model_type': result.get('model_type', 'N/A')
            }
            rows.append(row)
        
        # Add price target predictions
        if 'price_targets' in self.results:
            for period, data in self.results['price_targets'].items():
                for pct_change, pred_data in data['predictions'].items():
                    row = {
                        'prediction_type': f'price_target_{period}',
                        'target_pct': pct_change,
                        'target_price': pred_data['target_price'],
                        'probability': pred_data['probability'],
                        'historical_success': pred_data['historical_success_rate'],
                        'confidence': pred_data['confidence'],
                        'target_date': data['target_date']
                    }
                    rows.append(row)
        
        df = pd.DataFrame(rows)
        df.to_csv(filename, index=False)
    
    def display_results(self):
        """Display formatted results with enhanced explanations"""
        print(f"\n{'='*80}")
        print(f"MULTI-MODEL STOCK PREDICTIONS - {self.symbol}")
        print(f"Current Price: ${self.current_price:.2f}")
        print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'='*80}")
        
        # Main predictions with explanations
        explanations = {
            'direction': {
                'desc': 'Overall 5-day price direction prediction',
                'high_threshold': 0.6,
                'low_threshold': 0.4
            },
            'volatility': {
                'desc': 'Likelihood of volatility spike in next 3 days',
                'high_threshold': 0.7,
                'low_threshold': 0.3
            },
            'mean_reversion': {
                'desc': 'Probability of return to 20-day moving average',
                'high_threshold': 0.6,
                'low_threshold': 0.4
            },
            'momentum': {
                'desc': 'Likelihood current trend continues for 5 more days',
                'high_threshold': 0.65,
                'low_threshold': 0.35
            },
            'implied_move': {
                'desc': 'Probability of larger-than-normal price movement',
                'high_threshold': 0.6,
                'low_threshold': 0.4
            }
        }
        
        print(f"\n📈 INDIVIDUAL MODEL PREDICTIONS:")
        print("-" * 80)
        
        for name, result in self.results.items():
            if name in ['summary', 'ensemble', 'price_targets']:
                continue
                
            exp = explanations.get(name, {})
            desc = exp.get('desc', 'Model prediction')
            
            print(f"\n📊 {name.upper()}: {desc}")
            if 'prediction' in result:
                prob = result['prediction']
                print(f"  Probability: {prob:.3f} ({prob*100:.1f}%)")
                
                # Add interpretation
                if prob > exp.get('high_threshold', 0.6):
                    interpretation = "🟢 STRONG signal - High confidence"
                elif prob > 0.5:
                    interpretation = "🟡 MODERATE signal - Some confidence"
                elif prob < exp.get('low_threshold', 0.4):
                    interpretation = "🔴 STRONG opposite signal - High confidence"
                else:
                    interpretation = "⚪ WEAK signal - Low confidence"
                
                print(f"  Interpretation: {interpretation}")
            
            if 'signal' in result:
                print(f"  Trading Signal: {result['signal']}")
            if 'accuracy' in result:
                print(f"  Historical Accuracy: {result['accuracy']:.3f} ({result['accuracy']*100:.1f}%)")
            if 'model_type' in result:
                print(f"  Model Used: {result['model_type']}")
        
        # Ensemble with detailed explanation
        if 'ensemble' in self.results:
            ens = self.results['ensemble']
            print(f"\n🎯 ENSEMBLE FORECAST (Combined Analysis):")
            print("-" * 50)
            print(f"  Combined Prediction: {ens['ensemble_prediction']:.3f} ({ens['ensemble_prediction']*100:.1f}%)")
            print(f"  Model Agreement: {(1-ens['confidence']):.3f} (Higher = more models agree)")
            print(f"  Final Signal: {ens['signal']}")
            
            # Explain ensemble signal
            ensemble_explanations = {
                'STRONG_BUY': '🟢 Multiple models strongly bullish - Consider buying',
                'BUY': '🟢 Models lean bullish - Cautious buy signal',
                'HOLD': '🟡 Models mixed or neutral - Wait for clearer signal',
                'SELL': '🔴 Models lean bearish - Consider selling',
                'STRONG_SELL': '🔴 Multiple models strongly bearish - Strong sell signal'
            }
            
            explanation = ensemble_explanations.get(ens['signal'], 'Mixed signals')
            print(f"  Meaning: {explanation}")
        
        # Price target predictions
        if 'price_targets' in self.results:
            self.display_price_targets()
        
        # Risk disclaimer
        print(f"\n⚠️  IMPORTANT DISCLAIMERS:")
        print("• These are probabilistic predictions based on historical patterns")
        print("• Past performance does not guarantee future results")
        print("• Consider multiple factors and consult financial advisors")
        print("• Models may fail during unprecedented market conditions")
    
    def display_price_targets(self):
        """Display specific price target predictions in a formatted table"""
        if 'price_targets' not in self.results:
            return
        
        print(f"\n💰 SPECIFIC PRICE TARGET PREDICTIONS:")
        print("="*80)
        
        for period, data in self.results['price_targets'].items():
            period_name = "2 Weeks" if period == "2_weeks" else "4 Weeks"
            print(f"\n📅 {period_name} Target (by {data['target_date']}):")
            print("-" * 70)
            print(f"{'Target':>8} {'Price':>10} {'Probability':>12} {'Historical':>12} {'Confidence':>11}")
            print("-" * 70)
            
            # Sort by percentage change
            sorted_predictions = sorted(data['predictions'].items())
            
            for pct_change, pred_data in sorted_predictions:
                prob = pred_data['probability']
                hist = pred_data['historical_success_rate']
                conf = pred_data['confidence']
                price = pred_data['target_price']
                
                # Color coding for probability
                if prob > 70:
                    indicator = "🟢"
                elif prob > 55:
                    indicator = "🟡"
                elif prob < 30:
                    indicator = "🔴"
                else:
                    indicator = "⚪"
                
                print(f"{indicator} {pct_change:+3}% ${price:>8.2f} {prob:>9.1f}% {hist:>9.1f}% {conf:>9.1f}")
            
            # Highlight best opportunities
            best_up = max([p for p in sorted_predictions if p[0] > 0], 
                         key=lambda x: x[1]['probability'], default=None)
            best_down = max([p for p in sorted_predictions if p[0] < 0], 
                           key=lambda x: x[1]['probability'], default=None)
            
            print(f"\n  🎯 Highest Probability Moves:")
            if best_up:
                print(f"    Upward: {best_up[0]:+}% ({best_up[1]['probability']:.1f}% chance)")
            if best_down:
                print(f"    Downward: {best_down[0]:+}% ({best_down[1]['probability']:.1f}% chance)")
            
            # Risk assessment
            total_prob_up = sum([pred['probability'] for pct, pred in sorted_predictions if pct > 0])
            total_prob_down = sum([pred['probability'] for pct, pred in sorted_predictions if pct < 0])
            
            print(f"\n  📊 Directional Bias:")
            print(f"    Upward bias: {total_prob_up/5:.1f}% (avg of upward targets)")
            print(f"    Downward bias: {total_prob_down/5:.1f}% (avg of downward targets)")

# GCP Optimization Functions (Additional 50 lines)
def setup_gcp_logging():
    """Setup Google Cloud Logging"""
    try:
        from google.cloud import logging as cloud_logging
        client = cloud_logging.Client()
        client.setup_logging()
        return client
    except ImportError:
        logger.info("Google Cloud Logging not available, using standard logging")
        return None

def save_to_gcs(data, bucket_name, blob_name):
    """Save data to Google Cloud Storage"""
    try:
        from google.cloud import storage
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        blob.upload_from_string(json.dumps(data, default=str))
        logger.info(f"Data saved to gs://{bucket_name}/{blob_name}")
    except ImportError:
        logger.info("Google Cloud Storage not available")

def cloud_function_handler(request):
    """Google Cloud Function entry point"""
    try:
        request_json = request.get_json(silent=True)
        symbol = request_json.get('symbol', 'AAPL') if request_json else 'AAPL'
        
        predictor = MultiStockPredictor(symbol=symbol)
        results = predictor.run_all_predictions()
        
        # Save to GCS if configured
        bucket_name = os.environ.get('GCS_BUCKET')
        if bucket_name:
            blob_name = f"predictions/{symbol}_{datetime.now().strftime('%Y%m%d_%H%M')}.json"
            save_to_gcs(results, bucket_name, blob_name)
        
        return results
    except Exception as e:
        logger.error(f"Cloud function error: {e}")
        return {'error': str(e)}

def batch_predict_multiple_stocks(symbols, max_workers=3):
    """Batch prediction for multiple stocks (GCP optimized)"""
    import concurrent.futures
    
    def predict_single(symbol):
        predictor = MultiStockPredictor(symbol=symbol)
        return symbol, predictor.run_all_predictions()
    
    results = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(predict_single, symbol): symbol for symbol in symbols}
        
        for future in concurrent.futures.as_completed(futures):
            symbol = futures[future]
            try:
                symbol, result = future.result()
                results[symbol] = result
            except Exception as e:
                logger.error(f"Error predicting {symbol}: {e}")
                results[symbol] = {'error': str(e)}
    
    return results

# Main execution
if __name__ == "__main__":
    # Setup GCP logging if available
    gcp_logger = setup_gcp_logging()
    
    # Single stock prediction
    SYMBOL = "AAPL"
    
    predictor = MultiStockPredictor(symbol=SYMBOL)
    results = predictor.run_all_predictions()
    
    predictor.display_results()
    predictor.save_results()
    
    # Multi-stock example
    # symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA']
    # batch_results = batch_predict_multiple_stocks(symbols)
    # print(f"Batch prediction completed for {len(batch_results)} stocks")

INFO:__main__:Google Cloud Logging not available, using standard logging
INFO:__main__:Initialized predictor for AAPL
INFO:__main__:Fetching data for AAPL
INFO:__main__:Current AAPL price: $238.15
INFO:__main__:Running all prediction models...
INFO:__main__:Training price direction prediction model...
INFO:__main__:Training volatility breakout prediction...
INFO:__main__:Training mean reversion prediction...
INFO:__main__:Training momentum continuation prediction...
INFO:__main__:Training implied move prediction...
INFO:__main__:Training specific price target predictions...
INFO:__main__:Results saved to AAPL_predictions_20250916_1814.json and AAPL_predictions_20250916_1814_detailed.csv



MULTI-MODEL STOCK PREDICTIONS - AAPL
Current Price: $238.15
Analysis Date: 2025-09-16 18:14:44

📈 INDIVIDUAL MODEL PREDICTIONS:
--------------------------------------------------------------------------------

📊 DIRECTION: Overall 5-day price direction prediction
  Probability: 0.360 (36.0%)
  Interpretation: 🔴 STRONG opposite signal - High confidence
  Trading Signal: SELL
  Historical Accuracy: 0.510 (51.0%)
  Model Used: direction_classifier

📊 VOLATILITY: Likelihood of volatility spike in next 3 days
  Probability: 0.046 (4.6%)
  Interpretation: 🔴 STRONG opposite signal - High confidence
  Trading Signal: NORMAL_VOL
  Model Used: volatility_lstm

📊 MEAN_REVERSION: Probability of return to 20-day moving average
  Probability: 0.102 (10.2%)
  Interpretation: 🔴 STRONG opposite signal - High confidence
  Trading Signal: NO_REVERSION
  Model Used: mean_reversion_gb

📊 MOMENTUM: Likelihood current trend continues for 5 more days
  Probability: 0.100 (10.0%)
  Interpretation: 🔴 STRONG op